# Plotting Traffic Data and Shelters

In [1]:
# Loading libraries
import geopandas as gpd
import pandas as pd
import plotly.express as px
import sys
import os

In [2]:
# Load the shelter data
shelter_data  = gpd.read_file(os.path.join('..','data','shelters23.geojson')) 

# Load the real-time movement data 
traffic_data = pd.read_csv('b3eeb0ff-c8a8-4824-99d6-e0a3747c8b0d.csv') # Accessed from OpenData
road_data = pd.read_csv('c3097987-c394-4092-ad1d-ad86a81dbf37.csv') # Accessed from OpenData

In [3]:
# merge traffic and road data
merged_data = pd.merge(traffic_data, road_data, on='_id')

# filter to only get those with > 0 vehicle count and below 70 (to remove outliers outside of aarhus c)
merged_data = merged_data[merged_data['vehicleCount'] > 0]
merged_data = merged_data[merged_data['vehicleCount'] < 70]

In [4]:
# Create a DataFrame with the required columns
map_data = merged_data[['POINT_2_LAT', 'POINT_2_LNG', 'vehicleCount']].copy()

In [6]:
# Create a DataFrame with the required columns for traffic data
map_data = merged_data[['POINT_2_LAT', 'POINT_2_LNG', 'vehicleCount']].copy()
map_data.columns = map(str.lower, map_data.columns)

# Create a DataFrame for shelters with latitude and longitude
shelter_data['lat'] = shelter_data.geometry.y
shelter_data['lng'] = shelter_data.geometry.x

# Create a scatter mapbox plot for traffic data
fig = px.scatter_mapbox(
    map_data,
    lat="point_2_lat",
    lon="point_2_lng",
    size="vehiclecount",
    color="vehiclecount",
    hover_name="vehiclecount",
    zoom=12,
    height=800,
    width=1200,
    mapbox_style="carto-positron",
    title="Traffic Data (2024-03-22 07:15:00) and Shelters."
)

# Add shelter data to the map
fig.add_trace(px.scatter_mapbox(
    shelter_data,
    lat="lat",
    lon="lng",
    hover_name="identifier"
).data[0])

# Set your Mapbox token
fig.update_layout(mapbox_accesstoken="your-token")

# Show the plot
fig.show()